## Importing required libraries

In [103]:
from numpy.linalg import matrix_rank
import numpy as np
import pickle
import itertools
import numpy.linalg as la
import scipy.linalg as spla
import sys
import random
import time
import sklearn
from sklearn import datasets
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve

# for finite field calculation
from sympy.matrices import *

k_dict = {}
# row_order,col_order = (40,30) 
# with open(f'{row_order}x{col_order}.pkl','rb') as fp:
#     k_dict = pickle.load(fp)

## Get signature of a vector

In [155]:
def get_original_matrix():
    np.random.seed(0)
    L = np.array(np.random.randint(5,size=(100,5)),dtype=float)
    R = np.array(np.random.randint(5,size=(30,5)),dtype=float)
    A = L.dot(R.T)%5
    return A, L.dot(R.T)
A, A_ = get_original_matrix()
print(f'A:{matrix_rank(A)}, A:{matrix_rank(A_)}')
A

A:30, A:5


array([[1., 2., 0., ..., 1., 1., 3.],
       [2., 1., 1., ..., 3., 1., 2.],
       [1., 3., 2., ..., 2., 1., 3.],
       ...,
       [4., 3., 4., ..., 2., 1., 1.],
       [4., 0., 4., ..., 1., 3., 4.],
       [0., 2., 3., ..., 3., 2., 1.]])

### Function for row and col indices for missing values

In [106]:
# for a vector
def sgn(x): 
    return [i[0] for i in np.argwhere(np.isnan(x))]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# for a matrix
def signature(Y): 
    if np.count_nonzero(np.isnan(Y)) == 0:
        return [],[]
    else:
        missing_row_indices, missing_col_indices = zip(*np.argwhere(np.isnan(Y)))
        missing_row_indices,missing_col_indices = list(set(missing_row_indices)), list(set(missing_col_indices))
        return missing_row_indices, missing_col_indices

## FPT Algorithm

In [156]:
                                                # MATRIX PARAMETERS

# Building an incomplete matrix, A of order m*n
row_order,col_order = (10,8)   

# Sampling probability
prob = 0.2

#Given rank
r = 5
rank = 5

# GF(p)
p = 5

# for plotting purposes
if f'{row_order}x{col_order}' not in k_dict.keys():
    k_dict[f'{row_order}x{col_order}']={}
    k_dict[f'{row_order}x{col_order}']['row_order'] = row_order
    k_dict[f'{row_order}x{col_order}']['col_order'] = col_order
    k_dict[f'{row_order}x{col_order}']['prob'] = []
    k_dict[f'{row_order}x{col_order}']['#missing_entries'] = []
    k_dict[f'{row_order}x{col_order}']['k'] = []
    k_dict[f'{row_order}x{col_order}']['r'] = []
    k_dict[f'{row_order}x{col_order}']['FPTtime'] = []
    k_dict[f'{row_order}x{col_order}']['ConvexTime'] = []

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Function for getting the original matrix and random samples
def get_original_matrix():
    np.random.seed(0)
    L = np.array(np.random.randint(p,size=(row_order,rank)),dtype=float)
    R = np.array(np.random.randint(p,size=(col_order,rank)),dtype=float)
    A = L.dot(R.T)%p
    random.seed(0)
    samples = random.sample([(i, j) for i in range(5) for j in range(col_order-1)], int(row_order*col_order*prob))
    return A,samples
 #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
A,samples = get_original_matrix()
k_dict[f'{row_order}x{col_order}']['prob'].append(prob)

print(f'Rank of A: {matrix_rank(A)} | Shape: {A.shape}')
#Putting NaNs after sampling the matrix
for i,j in samples:
    A[i,j] = np.nan


# number of rows containing missing entries
k,_ = signature(A)   
k = len(k)
k_dict[f'{row_order}x{col_order}']['k'].append(k)


# starting time
start = time.time()
#****************************************************************************
                                 
def MatrixCompletion(A,r):
    
#     print('Matrix A:\n',A,end='\n\n')
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                                # Defining Functions
    
    #use function defined in previous cell
    compl_M, compl_N = signature(A) 
    M,N = [i for i in range(row_order) if i not in compl_M], [i for i in range(col_order) if i not in compl_N]

#     print(f'Non-Missing indices, M:{M}, N:{N} \n')
    #number of rows containing missing entries

    # M,N

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                                       # Checking for No instance
    if (matrix_rank(A[M]) > r) or (matrix_rank(A[:,N])) > r:
        print ('Oops!This Matrix is a no instance\n', A)
        return A
#     else:
#         print ('It is safe to move ahead with the algorithm\n')


    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                                        # Algorithm 1: Line 6-20 
    B = A[:,N]
#     print(f'Matrix B and compl_N:{compl_N} :\n',B)

    for j in compl_N:
        S = sgn(A[:,j])
        compl_S = [i for i in range(row_order) if i not in S]   ## A[np.ix_(M,N)] for calculating submatrices
        
#         print(f'Inside for loop S:{S}\n')
#         print(f'Inside for loop compl_S:{compl_S}\n')
#         print('Column space checking: \n', np.c_[(B[compl_S,:], A[compl_S,j])],'\n')
#         print(B[compl_S,:],'\n')
#         print(A[compl_S,j],'\n')
                  
        # Checking if the vector lies in column space of B
        if matrix_rank(np.c_[(B[compl_S,:], A[compl_S,j])]) == matrix_rank(B[compl_S,:]):
            
            #Solving system of linear equations
            
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
                                 #Normal LinAlg method
#             v = np.linalg.solve(B[compl_S,:], A[compl_S,j])
#             Q, R = la.qr(B[compl_S,:])
#             v = spla.solve_triangular(R, Q.T.dot(A[compl_S,j]), lower=False)
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
                                # Sage mode
#             sympy_A, sympy_b = Matrix(B[compl_S,:]), Matrix(A[compl_S,j])
#             v = sympy_A.inv_mod(p).dot(sympy_b)
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
                                # Using Sparse matrices
            sparse_A, sparse_b = csc_matrix(B[compl_S,:]), csc_matrix(A[compl_S,j])
            v = spsolve(sparse_A, sparse_b.T)
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

            #filling missing values with obtained solution
            sparse_B, sparse_v = csc_matrix(B[S,:]), csc_matrix(v)
            print(f'sparse A & b: {sparse_A.toarray()} -------- {sparse_b.T.toarray()}')
            print(f'sparse v: {sparse_v.toarray()}')
            print(f'sparse_B: {sparse_B.toarray()}')
            A[S,j] = (sparse_B.dot(sparse_v.T)).reshape((sparse_B.shape[0],)).toarray()
#             A[S,j] = B[S,:].dot(v)
            
            

    #getting new values for N
    _, compl_N = signature(A) 
    N = [i for i in range(col_order) if i not in compl_N]
    print(f'Updated Non-missing column indices: {N}')
    
    
    #updating B with new values of N
    B = A[:,N]

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    if compl_N == []:
#         print('We have the filling and that is:\n', A)
        return A
    else:
        if matrix_rank(B)==r:
#             print('Matrix B:\n',B)
            print ('Sorry! Matrix A is a no instance \n', A)
            return A            
            
            
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                                # Algorithm 1: Line 20-32

    index = compl_N[0]
#     print(f'After basis check, missing entries: {compl_N} \n')
    S = sgn(A[:,index])

    # Get all combinations to branch over
    all_combinations = [p for p in itertools.product(list(range(p)), repeat=len(S))]
#     print(all_combinations)
    
    for v in all_combinations:
        C = A
        C[S,index] = v
        
#         print('Matrix C: \n',C,'\n\n')
        
        #filling returned
        A_ = MatrixCompletion(C,r)
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        if matrix_rank(A_)<= r:
            print('The optimal filling that exists is:\n', A_)
            return A
        else:
            print ('Given Matrix A is a no instance\n',A)
            return A
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
MatrixCompletion(A,r)
# #ending time
end = time.time()
k_dict[f'{row_order}x{col_order}']['FPTtime'].append(end-start)

Rank of A: 8 | Shape: (10, 8)
Updated Non-missing column indices: [7]
Updated Non-missing column indices: [0, 7]
Updated Non-missing column indices: [0, 1, 7]
Updated Non-missing column indices: [0, 1, 2, 7]
Oops!This Matrix is a no instance
 [[ 3.  0.  0.  2. nan  3. nan  4.]
 [ 1.  3.  0.  2. nan nan  2.  3.]
 [ 2.  0.  0.  4. nan  1.  4.  2.]
 [ 2.  2.  4.  0. nan nan  4.  3.]
 [ 0.  0.  0.  0.  4.  3.  3.  0.]
 [ 3.  2.  2.  3.  4.  1.  0.  0.]
 [ 1.  0.  3.  4.  3.  1.  4.  3.]
 [ 0.  4.  2.  1.  4.  1.  0.  3.]
 [ 4.  3.  1.  3.  0.  1.  1.  3.]
 [ 1.  3.  2.  4.  3.  3.  2.  0.]]
The optimal filling that exists is:
 [[ 3.  0.  0.  2. nan  3. nan  4.]
 [ 1.  3.  0.  2. nan nan  2.  3.]
 [ 2.  0.  0.  4. nan  1.  4.  2.]
 [ 2.  2.  4.  0. nan nan  4.  3.]
 [ 0.  0.  0.  0.  4.  3.  3.  0.]
 [ 3.  2.  2.  3.  4.  1.  0.  0.]
 [ 1.  0.  3.  4.  3.  1.  4.  3.]
 [ 0.  4.  2.  1.  4.  1.  0.  3.]
 [ 4.  3.  1.  3.  0.  1.  1.  3.]
 [ 1.  3.  2.  4.  3.  3.  2.  0.]]
The optimal fillin

/home/navish/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1881: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)


array([[ 3.,  0.,  0.,  2., nan,  3., nan,  4.],
       [ 1.,  3.,  0.,  2., nan, nan,  2.,  3.],
       [ 2.,  0.,  0.,  4., nan,  1.,  4.,  2.],
       [ 2.,  2.,  4.,  0., nan, nan,  4.,  3.],
       [ 0.,  0.,  0.,  0.,  4.,  3.,  3.,  0.],
       [ 3.,  2.,  2.,  3.,  4.,  1.,  0.,  0.],
       [ 1.,  0.,  3.,  4.,  3.,  1.,  4.,  3.],
       [ 0.,  4.,  2.,  1.,  4.,  1.,  0.,  3.],
       [ 4.,  3.,  1.,  3.,  0.,  1.,  1.,  3.],
       [ 1.,  3.,  2.,  4.,  3.,  3.,  2.,  0.]])

## Optimal Filling obtained after our algorithm

In [157]:
print(f"Optimal filling of Rank: {matrix_rank(A)}\n {A}")
k_dict[f'{row_order}x{col_order}']['r'].append(matrix_rank(A))

Optimal filling of Rank: 0
 [[ 3.  0.  0.  2. nan  3. nan  4.]
 [ 1.  3.  0.  2. nan nan  2.  3.]
 [ 2.  0.  0.  4. nan  1.  4.  2.]
 [ 2.  2.  4.  0. nan nan  4.  3.]
 [ 0.  0.  0.  0.  4.  3.  3.  0.]
 [ 3.  2.  2.  3.  4.  1.  0.  0.]
 [ 1.  0.  3.  4.  3.  1.  4.  3.]
 [ 0.  4.  2.  1.  4.  1.  0.  3.]
 [ 4.  3.  1.  3.  0.  1.  1.  3.]
 [ 1.  3.  2.  4.  3.  3.  2.  0.]]


/home/navish/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1881: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)


### Candes and Recht algorithm

In [158]:
import math
import itertools
from cvxpy import *

# Noting starting time
start = time.time()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def _solve(M, omega):
    n_1 = M.shape[0]
    n_2 = M.shape[1]
    X_ = Semidef(n_1 + n_2)
    objective = Minimize(trace(X_))

    constraints = [(X_ == X_.T)]  # add symmetric constraint.
    for i, j in omega:
        constr = (X_[i, j + n_1] == M[i, j])
        constraints.append(constr)
    problem = Problem(objective, constraints)
    problem.solve(solver=CVXOPT)

    print("STATUS       :", problem.status)
    print("OPTIMAL VALUE:", problem.value)

    X0 = X_.value
    # check optimizer's solution is symmetric.
    print("|X0-X0.T|_F  :", np.linalg.norm(np.subtract(X0, X0.T), "fro"))
    return X_.value[:n_1, n_1:]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def _get_mask_matrix(n_1, n_2, omega):
    """
    If we observed entry (i, j) of matrix M, the entry of mask matrix is 1,
    Otherwise 0.
    """
    mask = np.zeros((n_1, n_2), dtype=np.int8)
    for i, j in omega:
        mask[i, j] = 1
    return mask


def _get_abs_max_from_matrix(M):
    return np.max(np.absolute(M))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def main(n_1, n_2, r, m):
    print("#row of M    :", n_1)
    print("#column of M :", n_2)
    print("#sample      :", n_1*n_2-m)
    k_dict[f'{row_order}x{col_order}']['#missing_entries'].append(m)
    M,omega_compl = get_original_matrix()
    M_abs_max = _get_abs_max_from_matrix(M)
    # print("RANK of M    :", np.linalg.matrix_rank(M))
    print("|M|_*        :", np.linalg.norm(M, "nuc"))
    M_rank = np.linalg.matrix_rank(M)
    print("RANK of M    :", M_rank)
    U, S, V_T = np.linalg.svd(M, full_matrices=False)
    print(S)

    # getting samples
    omega = [(i, j) for i in range(n_1) for j in range(n_2) if (i,j) not in omega_compl]
    mask = _get_mask_matrix(n_1, n_2, omega)

    # M_ is for training data removing the data.
    # This block should not affect the result.
    # Just for defensive programming.
    # M_ = np.random.uniform(-M_abs_max, M_abs_max, M.shape)
    M_ = M.copy()
    np.place(M_, 1 - mask, M_abs_max * M_abs_max)
    X = _solve(M_, omega)
    X_rank = np.linalg.matrix_rank(X)
    print("RANK of X    :", X_rank)
    print("|X|_*        :", np.linalg.norm(X, "nuc"))
    U, S, V_T = np.linalg.svd(X, full_matrices=False)
    print(S)

    E = np.subtract(M, X)
    E_train = E.copy()
    np.place(E_train, 1 - mask, 0)
    print('TRAIN RMSE   :', np.linalg.norm(E_train, "fro") / m)
    E_test = E.copy()
    np.place(E_test, mask, 0)
    print('TEST  RMSE   :', np.linalg.norm(E_test, "fro") / (n_1 * n_2 - m))

    frobenius_norm_ratio = (
        np.linalg.norm(X - M, "fro") / np.linalg.norm(M, "fro"))
    print("|X-M|_F/|M|_F:", frobenius_norm_ratio)  # the metric of paper.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
if __name__ == '__main__':
    main(row_order,col_order, r=2, m=int(row_order * col_order * prob))
    
# Noting ending time
end = time.time()
k_dict[f'{row_order}x{col_order}']['ConvexTime'].append(end-start)

#row of M    : 10
#column of M : 8
#sample      : 64
|M|_*        : 48.239389557854075
RANK of M    : 8
[21.028337    6.14552252  5.47673391  4.91609227  4.08666166  3.38931138
  2.13745692  1.0592739 ]
STATUS       : optimal
OPTIMAL VALUE: 86.21546690065803
|X0-X0.T|_F  : 0.0
RANK of X    : 8
|X|_*        : 43.10773349706413
[2.05259456e+01 6.10125614e+00 5.83174845e+00 4.27246323e+00
 2.77824193e+00 2.69777893e+00 9.00299160e-01 2.26123248e-08]
TRAIN RMSE   : 1.2098374922832816e-16
TEST  RMSE   : 0.11599436969665751
|X-M|_F/|M|_F: 0.31148900069379


In [159]:
k_dict

{'10x5': {'row_order': 10,
  'col_order': 5,
  'prob': [0.2],
  '#missing_entries': [],
  'k': [5],
  'r': [5],
  'FPTtime': [],
  'ConvexTime': []},
 '10x10': {'row_order': 10,
  'col_order': 10,
  'prob': [0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
  '#missing_entries': [],
  'k': [5, 5, 5, 5, 5, 5],
  'r': [],
  'FPTtime': [],
  'ConvexTime': []},
 '10x8': {'row_order': 10,
  'col_order': 8,
  'prob': [0.2, 0.2, 0.2, 0.2, 0.2],
  '#missing_entries': [16],
  'k': [5, 5, 5, 5],
  'r': [8, 0],
  'FPTtime': [],
  'ConvexTime': [0.5836975574493408]}}

#### Saving the dict:

In [161]:
import pickle
with open(f'{row_order}x{col_order}.pkl','wb') as fp:
    pickle.dump(k_dict,fp, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'{row_order}x{col_order}.pkl','rb') as fp:
    k_dict = pickle.load(fp)